In [ ]:
%%capture
#export
from typing import *
from fastcore.all import *
from fastai2.basics import *

from transformers import PreTrainedTokenizer, AutoTokenizer

In [ ]:
# default_exp general

# General
>

## Tokenizer

In [ ]:
#export
class TransformersTokenizer():
    ''' fastai want the tokenizer can handle list of string.
        use in parallel_gen() '''
    def __init__(self, tokenizer: PreTrainedTokenizer):
        self.tokenizer = tokenizer
    def __call__(self, items: List[str]):
        return map(self.tokenizer.tokenize, items)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

texts = ['This is a test', 'Just test']
transfomersTokenizer = TransformersTokenizer(tokenizer)
tok_texts = list(transfomersTokenizer(texts))

test_eq(tok_texts, [['this', 'is', 'a', 'test'], ['just', 'test']])

In [ ]:
texts = ['This is a test', 'Just test']
# parallel_gen will return generator of (0, ['this', 'is', 'a', 'test']), (1, ['just', 'test'])
tok_texts = L(parallel_gen(TransformersTokenizer, texts, tokenizer=tokenizer)).sorted().itemgot(1)

test_eq(tok_texts, [['this', 'is', 'a', 'test'],['just', 'test']])

## Export - 

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_general.ipynb.
Converted 01a_transforms.ipynb.
Converted 01b_model_splits.ipynb.
Converted 01c_callbacks.ipynb.
Converted 99a_example_roberta_classification.ipynb.
Converted 99b_example_gpt2_lm.ipynb.
Converted index.ipynb.
